# Hand Gesture Detection System
---

## 1. Import necessary libraries

In [ ]:
import kagglehub
from google.colab import drive
import shutil
from pathlib import Path
import pandas as pd
import matplotlib.pyplot as plt
import cv2
import random
import os
from collections import Counter
import seaborn as sns

---

## 2. Download Dataset Files

In [ ]:
# Download the dataset from KaggleHub
src_path = kagglehub.dataset_download("marusagar/hand-gesture-detection-system")
print("Path to dataset files:", src_path)

---

## 3. Mount Google Drive

In [ ]:
# Mount Google Drive to access files
drive.mount('/content/drive')

---

## 4. Set Data Directory

In [ ]:
# Define the path to your dataset in Google Drive
DATA_DIR = "/content/drive/MyDrive/Datasets/hand-gesture-detection_dataset"

---

## 5. Data Organization Setup

In [ ]:
# Define source and target root paths
source_root = Path(src_path)
target_root = Path(DATA_DIR)

# Read and rename CSV columns
train_file = pd.read_csv(source_root / "train.csv")
val_file = pd.read_csv(source_root / "val.csv")

train_file.columns = ['Image Name']
val_file.columns = ['Image Name']

# Define class ID mapping
classes = {
    0: "Left_Swipe_new",
    1: "Right_Swipe_new",
    2: "Stop_new",
    3: "Thumbs_Down_new",
    4: "Thumbs_Up_new"
}

---

## 6. Organize Data Function

In [ ]:
def organize_data(csv_file, src_folder_name, target_subfolder):
    # Ensure target class folders exist
    for class_name in classes.values():
        (target_root / target_subfolder / class_name[:-4]).mkdir(parents=True, exist_ok=True)

    for _, row in csv_file.iterrows():
        folder_name = row['Image Name'][:row['Image Name'].find(";")]
        class_id = row['Image Name'][-1]
        class_name = classes[int(class_id)][:-4]

        src_folder = source_root / src_folder_name / src_folder_name / folder_name
        dest_folder = target_root / target_subfolder / class_name / folder_name

        if src_folder.exists():
            shutil.copytree(src_folder, dest_folder)
            print(f"Copied {folder_name} to {target_subfolder}/{class_name}")
        else:
            print(f"Folder {folder_name} not found in {src_folder_name}")

---

## 7. Run Data Organization

In [ ]:
# Organize training data
organize_data(train_file, "train", "train")

# Organize validation data
organize_data(val_file, "val", "val")

---

## 8. Create Test Split

In [ ]:
# Define paths
train_dir = Path(f"{DATA_DIR}/train")
test_dir = Path(f"{DATA_DIR}/test")

# Ensure test directory exists
test_dir.mkdir(parents=True, exist_ok=True)

# Move 4 samples from each class in train to test
for class_folder in train_dir.iterdir():
    if class_folder.is_dir():
        target_class_folder = test_dir / class_folder.name
        target_class_folder.mkdir(parents=True, exist_ok=True)

        samples = [f for f in class_folder.iterdir() if f.is_dir()]
        samples_to_move = samples[:4]

        for sample_folder in samples_to_move:
            dest = target_class_folder / sample_folder.name
            shutil.move(sample_folder, dest)
            print(f"Moved {sample_folder.name} to test/{class_folder.name}")

---

## 9. Visualize Class Distribution

In [ ]:
# Count number of samples per class
train_path = f"{DATA_DIR}/train"
train_counts = Counter([
    label for folder in os.listdir(train_path)
    if os.path.isdir(os.path.join(train_path, folder))
    for label in [folder] * len(os.listdir(os.path.join(train_path, folder)))
])

# Create DataFrame from counts
df = pd.DataFrame(train_counts.items(), columns=['Class', 'Count'])
df = df.sort_values(by='Count', ascending=False)

# Plot bar chart
plt.figure(figsize=(9, 5))
ax = sns.barplot(data=df, x='Class', y='Count', palette='viridis')
plt.title('Sample Count per Class – Train Set')

# Add count labels
for p in ax.patches:
    height = p.get_height()
    ax.text(
        p.get_x() + p.get_width() / 2,
        height / 2,
        f'{int(height)}',
        ha='center', va='center',
        color='white', fontsize=12
    )
plt.tight_layout()
plt.show()